# Task 1 # 

In [1]:
import pandas as pd
inf_data = pd.read_csv('/home/qwq/2021_ICM_Problem_D_Data/influence_data.csv')

In [2]:
s1 = set(inf_data['influencer_id'].drop_duplicates().reset_index(drop = True))
s2 = inf_data['follower_id'].drop_duplicates().reset_index(drop = True)
ids = list(s1.union(s2))
n = len(ids)
id2vid,vid2id = dict(),dict()
for i in range(n):
    id2vid[ids[i]] = i
    vid2id[i] = ids[i]
print('Number of nodes:')
print(n)

Number of nodes:
5603


In [3]:
edges = []
head = [[] for i in range(n)]
    
for idx,row in inf_data.iterrows():
    u,v = id2vid[row['influencer_id']],id2vid[row['follower_id']]
    edges.append((u,v))
    head[u].append(edges[len(edges) - 1])

print('Not multigraph:')
print(len(edges) == len(set(edges)))

Not multigraph:
True


In [4]:
print('Number of edges:')
print(len(edges))

Number of edges:
42770


In [5]:
from queue import Queue
def has_circle():
    in_d = [0 for i in range(n)]
    calculated = [0 for i in range(n)]
    for edge in edges:
        in_d[v] += 1
    q = Queue()
    cnt = 1
    while 1:
        more_nodes = 0
        for i in range(n):
            if in_d[i] == 0 and not calculated[i]:
                more_nodes = 1
                calculated[i] = 1
                q.put(i)
        if not more_nodes:
            break
        while not q.empty():
            x = q.get()
            cnt += 1
            for edge in head[x]:
                in_d[edge[1]] -= 1
    return cnt != n

print('Has circle:')
print(has_circle())

Has circle:
False


In [6]:
import numpy as np

def calculate_influence(gamma = 0.95):
    in_d,out_d = [0 for i in range(n)],[0 for i in range(n)]
    for edge in edges:
        in_d[edge[1]] += 1
        out_d[edge[0]] += 1
    inf = [0 if out_d[i] == 0 else -1 for i in range(n)]
    def dfs(x):
        if inf[x] != -1:
            return
        inf[x] = 0
        for edge in head[x]:
            dfs(edge[1])
            inf[x] += inf[edge[1]] / in_d[edge[1]]
        inf[x] = gamma * inf[x] + out_d[x]
        
    for u in range(n):
        if in_d[u] == 0:
            dfs(u)
    return inf

all_inf = calculate_influence()

In [7]:
start = [0 for i in range(n)]
for idx,row in inf_data.iterrows():
    start[id2vid[row['influencer_id']]] = row['influencer_active_start']
    start[id2vid[row['follower_id']]] = row['follower_active_start']

In [8]:
print('Correlation coefficient between influence value and active start:')
print(np.corrcoef(np.array(start),np.array(all_inf))[0][1])

Correlation coefficient between influence value and active start:
-0.25287147950809225


In [10]:
from tqdm.notebook import tqdm

sp = [[0 if j == i else float("inf") for j in range(n)] for i in range(n)]
def queue_bellmanford(s):
    inq = [0 for i in range(n)]
    inq[s] = 1
    q = Queue()
    q.put(s)
    while not q.empty():
        x = q.get()
        inq[x] = 0
        for edge in head[x]:
            v = edge[1]
            if sp[s][v] > sp[s][x] + 1:
                sp[s][v] = sp[s][x] + 1
                if not inq[v]:
                    inq[v] = 1
                    q.put(v)

for i in tqdm(range(n)):
    queue_bellmanford(i)
    

In [237]:
in_d = [0 for i in range(n)]
for edge in edges:
    in_d[edge[1]] += 1
        
def influence_2(u,v,gamma = 0.95):
    if sp[u][v] == float("inf") or u == v:
        return 0
    elif (u,v) in edges:
        return gamma * all_inf[v] / in_d[v] + 1
    else:
        return gamma ** sp[u][v] * (all_inf[v] / in_d[v])

In [11]:
def get_subnet(u,cnt,edges,head):
    n = len(head)
    vis = [0 for i in range(n)]
    head_r = [[] for i in range(n)]
    edges_r = []
    selected = []
    def bfs(cnt):
        q = Queue()
        vis[u] = 1
        q.put(u)
        while not q.empty() and cnt:
            x = q.get()
            vis[x] = 1
            selected.append(x)
            cnt -= 1
            for edge in head[x]:
                if not vis[edge[1]]:
                    q.put(edge[1])
    
    bfs(cnt)
    for x in selected:
        for edge in head[x]:
            if edge[0] in selected and edge[1] in selected:
                edges_r.append(edge)
                head_r[x].append(edge)
    return edges_r,head_r,selected

def get_name(vid):
    if vid2id[vid] in inf_data['influencer_id'].to_list():
        return inf_data[inf_data.influencer_id == vid2id[vid]]['influencer_name'].reset_index(drop = True)[0]
    return inf_data[inf_data.follower_id == vid2id[vid]]['follower_name'].reset_index(drop = True)[0]

edges_sub,head_sub,selected = None,None,None
root = 0
for i in range(n):
    if 5 < all_inf[i] < 10:
        print('Selected artist ID:')
        print(vid2id[i])
        print('Selected artist name:')
        print(get_name(i))
        root = i
        edges_sub,head_sub,selected = get_subnet(i,999,edges,head)
        print('All nodes in subnet:')
        print(selected)
        break

from graphviz import Digraph

dot = Digraph(comment = 'Subnet')
for node in selected:
    dot.node(str(node),get_name(node) + '\ninf=' + str(round(all_inf[node],2)) + ' in=' + str(in_d[node]),**{'width':'2', 'height':'1'})
for edge in edges_sub:
    dot.edge(str(edge[0]),str(edge[1]))
dot.format = 'png'
dot.render('output-graph.gv',view = True)

Selected artist ID:
393257
Selected artist name:
Newsboys
All nodes in subnet:
[7, 404, 3397, 3304, 1247, 4042, 1897, 2795]


'output-graph.gv.png'

In [14]:
print('inf_2 between Newsboys ans TobyMac:')
print(influence_2(root,id2vid[510885]))

inf_2 between Newsboys ans TobyMac:
1.2375


# Task 2

In [15]:
features = ['danceability','energy','valence','tempo','loudness','mode','key','acousticness','instrumentalness','liveness','speechiness','duration_ms'
]
class_features = ['mode','key']
art_data = pd.read_csv('/home/qwq/2021_ICM_Problem_D_Data/data_by_artist.csv')

In [218]:
def get_genre(i):
    if i in inf_data['influencer_id'].to_list():
        return inf_data[inf_data.influencer_id == i]['influencer_main_genre'].reset_index(drop = True)[0]
    elif i in inf_data['follower_id'].to_list():
        return inf_data[inf_data.follower_id == i]['follower_main_genre'].reset_index(drop = True)[0]
    return 'unknown'

def get_start(i):
    if i in inf_data['influencer_id'].to_list():
        return inf_data[inf_data.influencer_id == i]['influencer_active_start'].reset_index(drop = True)[0]
    elif i in inf_data['follower_id'].to_list():
        return inf_data[inf_data.follower_id == i]['follower_active_start'].reset_index(drop = True)[0]
    return -1

art_data['genre'] = None
art_data['start'] = None
for idx,row in tqdm(art_data.iterrows()):
    art_data.at[idx,'genre'] = get_genre(row['artist_id'])
    art_data.at[idx,'start'] = get_start(row['artist_id'])

In [21]:
print('Number of artists with an unknown main genre:')
print(len(art_data[art_data.genre == 'unknown']))

Number of artists with an unknown main genre:
252


In [22]:
fm_data = pd.read_csv('/home/qwq/2021_ICM_Problem_D_Data/full_music_data.csv')

In [219]:
def get_normalized(df):
    res = pd.DataFrame(df)
    for f in features:
        if f not in class_features:
            res[f] = (res[f] - res[f].mean()) / res[f].std()
    return res

art_norm = get_normalized(art_data)
fm_norm = get_normalized(fm_data)

In [101]:
from torch.utils.data import Dataset
import numpy as np

class MusicEmbedDataset(Dataset):
    def __init__(self):
        self.triplets = []
        for idx,row in tqdm(fm_norm.iterrows()):
            if len(eval(row['artists_id'])) > 1:
                continue
            m_vec = np.array(list(row[features]))
            a_vec = np.array(
                list(
                    art_norm[art_norm.artist_id == eval(row['artists_id'])[0]][features] \
                        .reset_index(drop = True).iloc(0)
                )
            ).squeeze(0)
            other_a_vec = np.array(
                list(
                    art_norm[art_norm.artist_id != eval(row['artists_id'])[0]][features].sample(1) \
                        .reset_index(drop = True).iloc(0)
                )
            ).squeeze(0)
            self.triplets.append((m_vec,a_vec,1))
            self.triplets.append((m_vec,other_a_vec,-1))
    
    def __len__(self):
        return len(self.triplets)
    
    def __getitem__(self,i):
        return self.triplets[i]

dataset = MusicEmbedDataset()

In [83]:
from torch import nn

class MLP(nn.Module):
    def __init__(self):
        super(MLP,self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(12,12),
            nn.ReLU(),
            nn.Linear(12,12),
            nn.ReLU(),
            nn.Linear(12,12)
        )
    
    def forward(self,x):
        return self.layers(x)

In [105]:
from torch.utils.data import DataLoader,random_split
import torch
from torch import optim

train_size = int(len(dataset) * 0.8)
val_size = len(dataset) - train_size
trainset,valset = random_split(dataset, [train_size,val_size])
n_epochs = 5
batch_size = 32
baseline = 0

model = MLP()

def train():
    loader = DataLoader(trainset,batch_size = batch_size,shuffle = True)
    model.train()
    optimizer = optim.Adam(model.parameters(),lr = 1e-3)
    criterion = nn.CosineEmbeddingLoss()
    for epoch in tqdm(range(n_epochs)):
        losses = []
        for idx,(x,y,l) in tqdm(enumerate(loader)):
            x,y,l = x.float(),y.float(),l.float()
            loss = criterion(model(x),model(y),l)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses.append(loss.cpu().detach().numpy())
        if epoch % 1 == 0:
            print('Epoch ' + str(epoch))
            print(np.mean(np.array(losses))) 
            test(model)
            model.train()

def calc_baseline():
    loader = DataLoader(valset,batch_size = batch_size,shuffle = True)
    criterion = nn.CosineEmbeddingLoss()
    sims = []
    for idx,(x,y,l) in enumerate(loader):
        x,y,l = x.float(),y.float(),l.float()
        sim = criterion(x,y,l)
        sims.append(sim)
    baseline = np.mean(np.array(sims))
    print('Raw loss:')
    print(baseline)
    
def test(model):
    loader = DataLoader(valset,batch_size = batch_size,shuffle = True)
    criterion = nn.CosineEmbeddingLoss()
    model.eval()
    sims = []
    for idx,(x,y,l) in enumerate(loader):
        x,y,l = x.float(),y.float(),l.float()
        x,y = model(x).detach(),model(y).detach()
        sim = criterion(x,y,l)
        sims.append(sim)
    print('Test result:')
    print(np.mean(np.array(sims)))

calc_baseline()
train()

Raw loss:
0.41326824



Epoch 0
0.26502213
Test result:
0.24457783



Epoch 1
0.24407102
Test result:
0.2419228



Epoch 2
0.24082513
Test result:
0.23944364



Epoch 3
0.23839658
Test result:
0.23818879



Epoch 4
0.23688029
Test result:
0.23704273



In [116]:
def sim(v1,v2):
    x,y = torch.from_numpy(v1).float(),torch.from_numpy(v2).float()
    x,y = model(x).detach().numpy(),model(y).detach().numpy()
    return np.sum(x * y) / (np.sum(x * x) ** 0.5 * np.sum(y * y) ** 0.5)

same_genre_sims = []
dif_genre_sims = []

art_part = art_data.sample(frac = 0.05)
for idx1,row1 in tqdm(art_part.iterrows()):
    for idx2,row2 in art_part.iterrows():
        if row1['genre'] != 'unknown' and row2['genre'] != 'unknown' and idx1 != idx2:
            s = sim(np.array(list(row1[features])),np.array(list(row2[features])))
            if row1['genre'] == row2['genre']:
                same_genre_sims.append(s)
            else:
                dif_genre_sims.append(s)


Mean simularities:
0.20070157232786645 0.05815185771226174


In [122]:
print('Number of simularities:')
print(len(same_genre_sims),len(dif_genre_sims))
print('Mean simularities:')
print(np.mean(np.array(same_genre_sims)),np.mean(np.array(dif_genre_sims)))
mx,mn = np.max(np.array(same_genre_sims + dif_genre_sims)),np.min(np.array(same_genre_sims + dif_genre_sims))

Number of simularities:
22236 56444
Mean simularities:
0.20070157232786645 0.05815185771226174


# Task 5

In [129]:
yr_data = pd.read_csv('/home/qwq/2021_ICM_Problem_D_Data/data_by_year.csv')
yr_norm = get_normalized(yr_data)

In [133]:
sims = []
for i in range(len(yr_data)):
    if i != 0:
        sims.append(sim(yr_norm[features].values[i - 1],yr_norm[features].values[i]))

In [191]:
import matplotlib.pyplot as plt
%matplotlib qt5

rev_years = [1929,1958,1961,1962,1969,1996]

plt.figure(figsize = (10, 10))
plt.xlabel('Year')
plt.ylabel('Similarity To Last Year')
plt.plot([i for i in range(1922,2021)],
         sims,
         marker = 'o',
         markevery = [i - 1922 for i in rev_years]
        )
plt.locator_params(nbins = 20)
for y in rev_years:
    plt.text(y + 0.7,sims[y - 1922],str(y))
plt.show()

In [221]:
top_artists = [[] for i in range(len(rev_years))]
for idx,row in tqdm(art_norm.iterrows()):
    for i in range(len(rev_years)):
        if row['start'] == -1 or row['start'] > rev_years[i] + 10:
            continue
        top_artists[i].append(
            (row['artist_id'],
             row['artist_name'],
             row['popularity'] * sim(art_norm[features].values[idx],
                                     yr_norm[features].values[rev_years[i] - 1921]
                                    )
            )
        )

In [222]:
rev_arts = []
top_k = 5
for a in top_artists:
    rev_arts.append(sorted(a,key = lambda x:x[2],reverse = True)[:top_k])

print('Revolution representatives:')
for i in range(len(rev_years)):
    print('Year ' + str(rev_years[i]))
    print(rev_arts[i])

Revolution representatives:
Year 1929
[(816895, 'The Delta Rhythm Boys', 53.07941797465448), (197532, 'Charles TrÃ©net', 37.969537800561795), (300659, 'Pedro Infante', 37.0950356961642), (591620, 'Antonio MachÃ\xadn', 36.15749852198499), (753094, 'Son House', 35.448002290608635)]
Year 1958
[(760977, 'The Five Stairsteps', 56.94247876480318), (863774, 'Ralph McTell', 55.0446090092075), (289177, 'Sam the Sham & the Pharaohs', 51.16187767683868), (773642, 'The Kingsmen', 50.3326057883514), (754032, 'The Beatles', 45.068159345223954)]
Year 1961
[(720786, 'Pete Brown', 43.732144307577265), (755030, 'Steve Harley & Cockney Rebel', 41.76825378098594), (6613, 'Ali Farka TourÃ©', 41.040785560799804), (866767, 'Randy VanWarmer', 40.986492064150994), (863774, 'Ralph McTell', 40.07940381391444)]
Year 1962
[(760977, 'The Five Stairsteps', 60.21656139816118), (773642, 'The Kingsmen', 56.346039802260165), (863774, 'Ralph McTell', 54.30191547691746), (752673, 'Steve Forbert', 54.12186447444841), (8667

In [235]:
rev_sims = [sims[y - 1922] for y in rev_years]
rev_year_datas = np.array([yr_norm[features].values[y - 1921] for y in rev_years])
print(np.vstack((rev_sims,rev_year_datas.transpose())))
corr_map = np.corrcoef(np.vstack((rev_sims,rev_year_datas.transpose())))
corr_map[0]

[[ 0.56265242  0.46231001  0.59072124  0.62201784  0.80514247  0.87449557]
 [ 2.10482871 -1.07432554 -0.92789473 -0.73572735 -0.90302098  1.04235132]
 [-1.30817064 -0.87742433 -0.68374648 -0.46292672  0.20481843  0.82568542]
 [ 1.74373552 -0.70331009 -0.31119532  0.40950885  0.34381092  0.47787888]
 [-0.81202059 -0.54872527 -0.64650449 -0.19158623  0.1616338   0.22687605]
 [-1.49763335 -0.75400332 -0.71594034 -0.36374332 -0.05316254  0.84200265]
 [ 1.          1.          1.          1.          1.          1.        ]
 [ 7.          0.          0.          0.          2.          7.        ]
 [ 0.13642012  0.79828548  0.70578529  0.5688225  -0.38602532 -0.94537277]
 [ 0.21061972 -0.04532997  0.52473362 -0.14374294 -0.27898927 -0.82755503]
 [ 1.46145282 -0.30269256  0.08461883 -0.13409938 -0.03876229 -0.48347773]
 [ 5.81838202 -0.65750394 -0.59154941 -0.65259281 -0.68270581 -0.20732922]
 [-2.25348709 -0.55302224  0.14663729 -0.65619511  0.64537039  0.83925891]]


array([ 1.        ,  0.18195458,  0.9204666 ,  0.21695366,  0.88096846,
        0.83702122,         nan,  0.45576963, -0.91639764, -0.73910463,
       -0.34651248, -0.24218257,  0.6850046 ])

| danceability | energy | valence | tempo | loudness | mode | key | acousticness | instrumentalness | liveness | speechness | duration_ms |
|  ----  | ----  | ----  | ----  | ----  | ----  | ----  | ----  | ----  | ----  | ----  | ----  |
| 0.18195458 | 0.9204666 | 0.21695366 | 0.88096846 | 0.83702122 |  nan | 0.45576963 | -0.91639764 | -0.73910463 | -0.34651248 | -0.24218257 |  0.6850046 |
| low | perfect | low | very large | very large | low | medium | perfect | very large | medium | low | high |


# Task 3

In [243]:
inf2_all = [[0 for i in range(n)] for i in range(n)]
for i in tqdm(range(n)):
    for j in range(n):
        inf2_all[i][j] = influence_2(i,j)

In [245]:
with open('inf2all.pkl','wb') as f:
    pkl.dump(inf2_all,f)

In [255]:
art_inf = pd.DataFrame(columns = ('id','name','genre','start'))
tmp = []
for idx,row in tqdm(inf_data.iterrows()):
    tmp.append({'id':row['influencer_id'],
                     'name':row['influencer_name'],
                     'genre':row['influencer_main_genre'],
                     'start':row['influencer_active_start']
                })
    tmp.append({'id':row['follower_id'],
                     'name':row['follower_name'],
                     'genre':row['follower_main_genre'],
                     'start':row['follower_active_start']
                })

In [256]:
art_inf = art_inf.append(tmp,ignore_index = True).drop_duplicates().reset_index(drop = True)
art_inf

,id,name,genre,start
0,759491,The Exploited,Pop/Rock,1980
1,74,Special Duties,Pop/Rock,1980
2,25462,Tricky,Electronic,1990
3,335,PJ Harvey,Pop/Rock,1990
4,66915,Bob Dylan,Pop/Rock,1960
...,...,...,...,...
5601,3661738,Rosemary & Garlic,Pop/Rock,2010
5602,261309,Vybz Kartel,Reggae,2000
5603,3670556,Trinidad Cardona,R&B;,2010
5604,2518003,Popcaan,Reggae,2000


In [344]:
genres = set(art_inf['genre'])
genres.remove('Unknown')
genres = list(genres)
genre_acnt = []
for g in genres:
    genre_acnt.append(len(art_inf[art_inf.genre == g]))
genres

['Easy Listening',
 'Classical',
 'International',
 'R&B;',
 'Latin',
 'New Age',
 'Religious',
 'Vocal',
 'Comedy/Spoken',
 'Stage & Screen',
 'Jazz',
 'Blues',
 'Pop/Rock',
 'Folk',
 'Country',
 'Reggae',
 'Avant-Garde',
 'Electronic',
 "Children's"]

In [345]:
import matplotlib.pyplot as plt

plt.figure(figsize = (20, 10))
plt.bar(genres,genre_acnt)
plt.xticks(range(len(genres)),genres,rotation = 45)
plt.xlabel('Genre')
plt.ylabel('Number of Artists')
plt.tight_layout()
plt.show()

In [346]:
genre_vecs = []
for g in genres:
    all_art_vec = art_norm[art_norm.artist_id.isin(
        list(art_inf[art_inf.genre == g]['id'])
    )][features].values
    genre_vec = np.zeros(len(features))
    for line in all_art_vec:
        genre_vec += line
    genre_vec /= len(all_art_vec)
    genre_vec[5] = int(genre_vec[5] + 0.5)
    genre_vec[6] = int(genre_vec[6] + 0.5) #Class features
    genre_vecs.append(genre_vec)

In [639]:
same_genre_infs = []
dif_genre_infs = []
same_genre_sims = []
dif_genre_sims = []

art_part = art_norm.sample(frac = 0.2)
for idx1,row1 in tqdm(art_part.iterrows()):
    for idx2,row2 in art_part.iterrows():
        if row1['genre'] != 'unknown' and row2['genre'] != 'unknown' and idx1 < idx2:
            s = sim(np.array(list(row1[features])),np.array(list(row2[features])))
            if row1['genre'] == row2['genre']:
                same_genre_sims.append(s)
            else:
                dif_genre_sims.append(s)
                
            a1,a2 = id2vid[row1['artist_id']],id2vid[row2['artist_id']]
            d = max(influence_2(a1,a2),influence_2(a2,a1))
            if row1['genre'] == row2['genre']:
                same_genre_infs.append(d)
            else:
                dif_genre_infs.append(d)

In [640]:
print('Miu and Sigma')
print('Same genre similarity:')
print(np.mean(np.array(same_genre_sims)),np.std(np.array(same_genre_sims)))
print('Different genre similarity:')
print(np.mean(np.array(dif_genre_sims)),np.std(np.array(dif_genre_sims)))
print('Same genre influence:')
print(np.mean(np.array(same_genre_infs)),np.std(np.array(same_genre_infs)))
print('Different genre influence:')
print(np.mean(np.array(dif_genre_infs)),np.std(np.array(dif_genre_infs)))

Miu and Sigma
Same genre similarity:
0.2359832994396561 0.44211757369971005
Different genre similarity:
0.02624870439475074 0.4398871857056197
Same genre influence:
0.3921641282325098 3.793201854918767
Different genre influence:
0.4804580181312725 2.764178424187365


In [641]:
same_genre_sigma,same_genre_miu = np.std(np.array(same_genre_infs)),np.mean(np.array(same_genre_infs))

In [642]:
art_df_list = []
for g in genres:
    if g != 'Unknown' and g != 'Children\'s':
        art_df_list.append(
            art_norm[art_norm.artist_id.isin(
                        list(art_inf[art_inf.genre == g]['id'])
            )].sample(n = 100,replace = True)
        )
art_df_list.append(art_norm[art_norm.artist_id.isin(
                        list(art_inf[art_inf.genre == 'Children\'s']['id'])
                )])
art_part = pd.concat(art_df_list).reset_index(drop = True)

In [643]:
genre_inf = [[0 for i in range(len(genres))] for i in range(len(genres))]
cnt = [[0 for i in range(len(genres))] for i in range(len(genres))]
for idx1,row1 in tqdm(art_part.iterrows()):
    for idx2,row2 in art_part.iterrows():
        if idx1 > idx2:
            a1,a2 = id2vid[row1['artist_id']],id2vid[row2['artist_id']]
            d = max(influence_2(a1,a2),influence_2(a2,a1))
            g1 = genres.index(list(art_inf[art_inf.id == row1['artist_id']]['genre'])[0])
            g2 = genres.index(list(art_inf[art_inf.id == row2['artist_id']]['genre'])[0])
            if g1 != g2:
                genre_inf[g1][g2] += d
                cnt[g1][g2] += 1

genre_related_prob = np.array(genre_inf) / np.array(cnt)

In [644]:
import scipy.stats as st

for i in range(len(genres)):
    for j in range(len(genres)):
        if genre_related_prob[i][j] == np.nan:
            genre_related_prob[i][j] = 0
        else:
            #genre_related_prob[i][j] = norm_prob_dense(genre_related_prob[i][j],same_genre_miu,same_genre_sigma)
            margin = abs(genre_related_prob[i][j] - same_genre_miu)
            genre_related_prob[i][j] = st.norm.cdf(same_genre_miu + margin,loc = same_genre_miu,scale = same_genre_sigma) - st.norm.cdf(same_genre_miu - margin,loc = same_genre_miu,scale = same_genre_sigma)

In [645]:
import matplotlib.pyplot as plt

plt.figure(figsize = (10, 10))
plt.imshow(genre_related_prob, cmap = plt.cm.cool)
plt.xticks(range(len(genres)),genres,rotation = 65)
plt.yticks(range(len(genres)),genres,rotation = 65)
plt.colorbar()
plt.show()

In [420]:
box_df = pd.DataFrame(np.array(genre_vecs),columns = features)
plt.figure(figsize = (10, 10))
box_df.boxplot(vert = False)
plt.show()

In [432]:
ti_n = 5
time_intervals = [[1921,1941],[1941,1961],[1961,1981],[1981,2001],[2001,2021]] #left closed right open

def get_active_artist_df(ti):
    return art_norm[(art_norm.start != -1) & \
                    (art_norm.start >= ti[0] - 10) & (art_norm.start < ti[1] + 10) & \
                    (art_norm.genre != 'unknown') & (art_norm.genre != 'Unknown')]

active_genres = []
for ti in time_intervals:
    active_genres.append(list(set(get_active_artist_df(ti)['genre'])))
print('Number of active genres in each time interval:')
for l in active_genres:
    print(len(l))

Number of active genres in each time interval:
17
19
19
19
17


In [458]:
genre_vecs_divided = [[] for i in range(ti_n)]

def get_genre_vec(g,ti):
    aadf = get_active_artist_df(ti)
    all_art_vec = aadf[aadf.genre == g][features].values
    genre_vec = np.zeros(len(features))
    for line in all_art_vec:
        genre_vec += line
    genre_vec /= len(all_art_vec)
    genre_vec[5] = int(genre_vec[5] + 0.5)
    genre_vec[6] = int(genre_vec[6] + 0.5) #Class features
    return genre_vec

for i in range(ti_n):
    for g in active_genres[i]:
        genre_vecs_divided[i].append(get_genre_vec(g,time_intervals[i]))

In [549]:
from sklearn.decomposition import PCA
from adjustText import adjust_text

pca = PCA(n_components = 2)
cur_ti = 0
g_vecs = pca.fit_transform(genre_vecs_divided[cur_ti])
plt.figure(figsize = (20, 20))
texts = []
for i in range(len(active_genres[cur_ti])):
    plt.scatter(g_vecs[i][0],g_vecs[i][1])
    texts.append(plt.text(g_vecs[i][0],g_vecs[i][1],active_genres[cur_ti][i]))
adjust_text(texts,only_move = {'text': 'xy'})
plt.show()

# Task 6

In [561]:
jazz_influencers = art_norm[art_norm.genre == 'Jazz'][['artist_id','artist_name','popularity']]
jazz_influencers['metric'] = None
for idx,row in jazz_influencers.iterrows():
    jazz_influencers.at[idx,'metric'] = all_inf[id2vid[row['artist_id']]] * row['popularity']
jazz_influencers.sort_values(by = 'metric',ascending = False).values[:10]

array([[532957, 'Cab Calloway', 27.66666667, 167301.63601383596],
       [287604, 'Louis Jordan', 35.0, 132106.34920063958],
       [259529, 'Lester Young', 12.02439024, 34001.45205368151],
       [317093, 'Nat King Cole', 24.41909814, 33611.41897732868],
       [805930, 'Charlie Christian', 10.37037037, 20930.254846077183],
       [423829, 'Miles Davis', 22.70023148, 13644.316049630726],
       [162677, 'Dizzy Gillespie', 13.88429752, 12194.149346353357],
       [175553, 'John Coltrane', 22.72727273, 11811.861361533154],
       [225389, 'Jay McShann', 47.0, 8762.66977449667],
       [211758, 'Charlie Parker', 16.97, 7823.994753611596]], dtype=object)

In [571]:
x_axis = ['1921~1940','1941~1960','1961~1980','1981~2000','2001~2020']
genre_selected = 'Jazz'
selected_vecs = []

for i in range(ti_n):
    selected_vecs.append(genre_vecs_divided[i][active_genres[i].index(genre_selected)])
selected_vecs = np.array(selected_vecs).transpose()

In [586]:
plt.figure(figsize = (10, 10))
plt.plot(x_axis, selected_vecs[0], color='green', label='danceability')
plt.plot(x_axis, selected_vecs[1], color='red', label='energy')
plt.plot(x_axis, selected_vecs[2],  color='skyblue', label='valence')
plt.plot(x_axis, selected_vecs[3], color='blue', label='tempo')
plt.plot(x_axis, selected_vecs[4], color='yellow', label='loudness')
plt.legend()
plt.show()

In [587]:
plt.figure(figsize = (10, 10))
plt.plot(x_axis, selected_vecs[7], color='green', label='acousticness')
plt.plot(x_axis, selected_vecs[8], color='red', label='instrumentalness')
plt.plot(x_axis, selected_vecs[9],  color='skyblue', label='liveness')
plt.plot(x_axis, selected_vecs[10], color='blue', label='speechiness')
plt.plot(x_axis, selected_vecs[11], color='yellow', label='duration_ms')
plt.legend()
plt.show()

In [588]:
plt.figure(figsize = (10, 10))
plt.plot(x_axis, selected_vecs[5], color='green', label='mode')
plt.plot(x_axis, selected_vecs[6], color='red', label='key')
plt.legend()
plt.show()

# Task 4

In [628]:
sims_without_edge = []
sims_with_edge = []

for idx1,row1 in tqdm(art_norm.sample(n = 300).iterrows()):
    for idx2,row2 in art_norm.sample(n = 300).iterrows():
        if row1['artist_id'] in id2vid and row2['artist_id'] in id2vid:
            u,v = id2vid[row1['artist_id']],id2vid[row2['artist_id']]
            if sp[u][v] != 1 and sp[v][u] != 1:
                sims_without_edge.append(sim(np.array(list(row1[features])),np.array(list(row2[features]))))

In [629]:
influencer_vecs = []
follower_vecs = []

for idx,row in tqdm(inf_data.iterrows()):
    inf_vec = np.array(art_norm[art_norm.artist_id == row['influencer_id']][features])
    if inf_vec.shape != (1,12):
        continue
    inf_vec = inf_vec.squeeze(0)
    fol_vec = np.array(art_norm[art_norm.artist_id == row['follower_id']][features])
    if fol_vec.shape != (1,12):
        continue
    fol_vec = fol_vec.squeeze(0)
    sims_with_edge.append(sim(inf_vec,fol_vec))
    influencer_vecs.append(inf_vec)
    follower_vecs.append(fol_vec)

In [630]:
sims_without_edge = np.array(sims_without_edge)
sims_with_edge = np.array(sims_with_edge)

print(np.mean(sims_without_edge),np.std(sims_without_edge))
print(np.mean(sims_with_edge),np.std(sims_with_edge))

0.09276848164100499 0.4490273710120095
0.5207397256161642 0.3841956255787107


In [627]:
selected_inf_idxs = sample([i for i in range(len(influencer_vecs))],k = 1000)

In [633]:
sims_feature_mask = [[] for i in range(len(features))]
for i in tqdm(range(len(features))):
    for idx in selected_inf_idxs:
        inf_vec = np.array(influencer_vecs[idx])
        fol_vec = np.array(follower_vecs[idx])
        inf_vec[i] = 0
        fol_vec[i] = 0
        sims_feature_mask[i].append(sim(inf_vec,fol_vec))

In [638]:
maksed_sim_mean_dev = [abs(np.mean(np.array(line)) - np.mean(sims_with_edge)) for line in sims_feature_mask]
plt.figure(figsize = (30, 10))
plt.plot(features,maksed_sim_mean_dev)
plt.show()

# Task 7

In [649]:
pop_by_year = yr_data[['year','popularity']].values.transpose()
plt.plot(pop_by_year[0],pop_by_year[1])
plt.xlabel('Year')
plt.ylabel('Popularity')
plt.show()

In [690]:
def get_active_artist_df(ti):
    return art_norm[(art_norm.start != -1) & \
                    (art_norm.start >= ti[0] - 30) & (art_norm.start < ti[1] + 10) & \
                    (art_norm.genre != 'unknown') & (art_norm.genre != 'Unknown')]

def genre_artist_plot(g):
    ti_n = 10
    tis = [[i,i + 10] for i in range(1921,2020,10)]
    x_axis = [str(ti[0] - 1) + 's' for ti in tis]
    y_axis = []
    for ti in tis:
        aadf = get_active_artist_df(ti)
        y_axis.append(np.sum(aadf[aadf.genre == g]['popularity'].values))
        #y_axis.append(len(aadf[aadf.genre == g]))
    plt.plot(x_axis,y_axis,label = g)

plt.figure(figsize = (10, 10))
plt.xlabel('Decade')
plt.ylabel('Total Polularity')
genre_artist_plot('Jazz')
genre_artist_plot('Reggae')
genre_artist_plot('Blues')
genre_artist_plot('R&B;')
genre_artist_plot('Electronic')
genre_artist_plot('Country')
plt.legend()
plt.show()